In [16]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct

client = QdrantClient(host="localhost", port=6333)

**Create collection**

In [30]:
collection_name = "aic2023-ocr-text"

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=4, distance=models.Distance.COSINE)
)

True

**Add vectors**

In [32]:
points = [
        PointStruct(id=0, vector=[0.05, 0.61, 0.76, 0.74]),
        PointStruct(id=1, vector=[0.19, 0.81, 0.75, 0.11]),
        PointStruct(id=2, vector=[0.36, 0.55, 0.47, 0.94]),
        PointStruct(id=3, vector=[0.18, 0.01, 0.85, 0.80])
    ]

operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,
    points=points,
)

print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


**Add payload**

In [33]:
ocr_text = ["Lasagne alla Bolognese", "Fettuccine al Pomodoro", "Gnocchi di Patate", "Melanzane alla Parmigiana"]

for i in range(4):
    payload = {
        "video_name": "",
        "keyframe_index": "",
        "ocr_text": ocr_text[i]
    }
    client.set_payload(
        collection_name=collection_name,
        payload=payload,
        points=[i]
    )

**Create payload index**

In [34]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="ocr_text",
    field_schema=models.TextIndexParams(
        type="text",
        tokenizer=models.TokenizerType.WORD,
        min_token_len=2,
        max_token_len=15,
        lowercase=True,
    )
)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
# Lấy thông tin
client.get_collection(collection_name=collection_name)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=4, indexed_vectors_count=0, points_count=4, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={'ocr_text': PayloadIndexInfo(data_type=<PayloadSchemaType.TEXT: 'text'>, params

In [37]:
# Now, let's perform a full-text search
query = "di patete"
search_results = client.search(collection_name="aic2023-ocr-text", query_vector=query)

# Print or process the search results as needed
print(search_results)

ValidationError: 2 validation errors for SearchRequest
vector.NamedVector
  Input should be a valid dictionary or instance of NamedVector [type=model_type, input_value='di patete', input_type=str]
    For further information visit https://errors.pydantic.dev/2.2/v/model_type
vector.list[float]
  Input should be a valid list [type=list_type, input_value='di patete', input_type=str]
    For further information visit https://errors.pydantic.dev/2.2/v/list_type